<a href="https://colab.research.google.com/github/Pickles91/DCAC/blob/main/Prostate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [ ]:
# check whether GPU accelerated computing is available
assert torch.cuda.is_available() # if there is an error here, enable GPU in the Runtime

In [ ]:
# install nnunet - yes it is that easy
!pip install nnunetv2

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
# install nnunet - yes it is that easy
!pip install nnunetv2

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
# for colab users only - mounting the drive

from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

Mounted at /content/drive


In [ ]:
assert os.path.exists(drive_dir) # if this fails, something went wrong with mounting GoogleDrive
if os.path.exists(mount_dir) is False:
    os.makedirs(mount_dir)

In [ ]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):

        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [ ]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw" : os.path.join(mount_dir, "nnUNet_raw"),
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    "nnUNet_results" : os.path.join(mount_dir, "nnUNet_results"),
    "RAW_DATA_PATH" : os.path.join(mount_dir, "RawData"), # This is used here only for convenience (not necessary for nnU-Net)!
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

Current Working Directory /content
/content/drive/My Drive/Colab Notebooks/nnUNet_raw created!
/content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed created!
/content/drive/My Drive/Colab Notebooks/nnUNet_results created!
/content/drive/My Drive/Colab Notebooks/RawData created!
If No Error Occured Continue Forward. =)


In [ ]:
print("Data should be located in folder: {}".format(path_dict["RAW_DATA_PATH"]))
assert os.path.isfile(os.path.join(path_dict["RAW_DATA_PATH"], "/content/drive/MyDrive/Colab Notebooks/RawData/Task05_Prostate.tar")) # check whether the file is correctly downloaded

Data should be located in folder: /content/drive/My Drive/Colab Notebooks/RawData


In [ ]:
!nnUNetv2_convert_MSD_dataset -i "${RAW_DATA_PATH}/Task05_Prostate"

In [ ]:
# Prepare the Execution of nnU-Net for Dataset 5
!nnUNetv2_plan_and_preprocess -d 5

Fingerprint extraction...
Dataset005_Prostate
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100% 20/20 [00:34<00:00,  1.73s/it]
Experiment planning...
Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [ 20. 320. 319.], 3d_lowres: [20, 320, 319]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 32, 'patch_size': (320, 320), 'median_image_size_in_voxels': array([320., 319.]), 'spacing': array([0.625, 0.625]), 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [False, False], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape', 'resampling_fn_data_kwargs': {'is_seg': False, 'order': 3, 'order_z': 0, 'force_separate_z': None}, 'resampling_fn_seg_kwargs': {'is_seg': True, 'order': 1, 'order_z': 0, 'force_separate_z': None}, 'resampling_

In [ ]:
# Download the Pretrained Model for the Prostate Dataset (Decathlon Dataset)
os.chdir(path_dict["nnUNet_results"])
!gdown 'https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy'
!nnUNetv2_install_pretrained_model_from_zip nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip
!rm 'nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip'
os.chdir(base_dir)

# takes roughly 9 minutes

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy
From (redirected): https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy&confirm=t&uuid=e307ec54-c343-43a9-b45e-30be1ae523e2
To: /content/drive/My Drive/Colab Notebooks/nnUNet_results/nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip
100% 1.66G/1.66G [00:37<00:00, 44.1MB/s]


In [ ]:
# use fully trained nnU-Net to make predictions on data
!nnUNetv2_predict -i "${nnUNet_raw}/Dataset005_Prostate/imagesTs/" -o "${nnUNet_results}/Dataset005_Prostate/predTs/" -d 5 -c 3d_fullres


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

/usr/local/lib/python3.10/dist-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
There are 16 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start c

In [ ]:
# Create Folderstructure for the new dataset!
dataset_name = 'Dataset501_SCGM' #change here for different dataset name
nnunet_raw_data = os.path.join(os.getenv("nnUNet_raw"))
task_folder_name = os.path.join(nnunet_raw_data,dataset_name)
train_image_dir = os.path.join(task_folder_name,'imagesTr')
train_label_dir = os.path.join(task_folder_name,'labelsTr')
test_dir = os.path.join(task_folder_name,'imagesTs')

# Create Folder Structure for the SCGM dataset on the system
make_if_dont_exist(task_folder_name)
make_if_dont_exist(train_image_dir)
make_if_dont_exist(train_label_dir)
make_if_dont_exist(test_dir)

training_data_name="training-data-gm-sc-challenge-ismrm16-v20160302b"
test_data_name="test-data-gm-sc-challenge-ismrm16-v20160401"

/content/drive/My Drive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM created!
/content/drive/My Drive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/imagesTr created!
/content/drive/My Drive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/labelsTr created!
/content/drive/My Drive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/imagesTs created!


In [ ]:
# only for in poerson workshops

os.chdir(task_folder_name)
# download training data
!gdown 'https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy'
!ls
os.chdir(base_dir)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy
From (redirected): https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy&confirm=t&uuid=cadf1339-23bd-4b2b-8258-dbc98f9778d6
To: /content/drive/My Drive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip
100% 1.66G/1.66G [00:24<00:00, 68.3MB/s]
imagesTr  imagesTs  labelsTr  nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip


In [ ]:
# continue from here again.
task_folder_name = "/content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip"

if os.path.isfile(task_folder_name):
    print("Training file for SCGM Challenge exists")
else:
    print("Please download the dataset zip file and place it at the following location: \n{}".format(task_folder_name))

Training file for SCGM Challenge exists


In [ ]:
#unzipping in nnUNet_raw folder the training data
# Specify the path to the zip file
zip_file_path = "/content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip"

# Specify the directory where you want to extract the contents of the zip file
extracted_folder_path = "/content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/"

# Execute the unzip command
!unzip "{zip_file_path}" -d "{extracted_folder_path}"

Archive:  /content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip
  inflating: /content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/Dataset005_Prostate/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/debug.json  
  inflating: /content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/Dataset005_Prostate/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/checkpoint_final.pth  
  inflating: /content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/Dataset005_Prostate/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/progress.png  
  inflating: /content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/Dataset005_Prostate/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/validation/summary.json  
  inflating: /content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/Dataset005_Prostate/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_1/debug.json  
  inflating: /content/drive/MyDrive/Colab Notebooks/nnUNet_r

In [ ]:
# function for copying, savind and renaming
def copy_and_rename(old_location,old_file_name,new_location,new_filename,delete_original = False):

    shutil.copy(os.path.join(old_location,old_file_name),new_location)
    os.rename(os.path.join(new_location,old_file_name),os.path.join(new_location,new_filename))
    if delete_original:
        os.remove(os.path.join(old_location,old_file_name))

# putting training images into folder

mask_count = 4 # change if more mask is available
base_data_folder_name = os.path.join(task_folder_name, "training-data-gm-sc-challenge-ismrm16-v20160302b")

for file in os.listdir(base_data_folder_name):
    # print(file)
    if file.endswith('.nii.gz'):
        if file.find('mask')!=-1:
            # putting mask
            shutil.move(os.path.join(base_data_folder_name,file),train_label_dir)
        else:
            # making 4 copies
            for mask in range(1,mask_count+1):
                new_filename = file[:file.find('-image')] + '-mask-r' + str(mask) + '.nii.gz'
                if mask==mask_count:
                    copy_and_rename(base_data_folder_name,file,train_image_dir,new_filename,delete_original = True)
                else:
                    copy_and_rename(base_data_folder_name,file,train_image_dir,new_filename)
    # removing all other files installed due to the unzip
    elif file.endswith('.txt'):
        os.remove(os.path.join(base_data_folder_name,file))

NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset501_SCGM/nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip/training-data-gm-sc-challenge-ismrm16-v20160302b'